# Lucene Scoring functions

## Classic Lucene Similarity: a modified TF-IDF

In information retrieval, [**tf–idf**](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), short for **term frequency–inverse document frequency**, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.

Lucene's Practical Scoring Function uses a modified version of the TF-IDF found in textbooks, whihc is explained below.

### Lucene modified TF

Instead of taking the TF as is, Lucene uses $\sqrt{TF}$. This way, documents with twice the number of terms as another document aren’t twice as relevant. Instead the TF score is computed as follows:

| TF | Lucene TF |
|------|------|
|   1  | 1 |
|   2  | 1.141 |
|   4  | 2 |
|   8  | 2.828 |
|   16  | 4 |


### Lucene modified IDF

Similarly users don’t consider terms that only occur in 10 documents ten times as special as those that occur in 100 documents. Instead, the IDF score is computed as:

$$ln\left ( \frac{docCount}{docFreq_{t} + 1} \right ) + 1$$

where *docCount* is the total number of documents in the collection, and *docFreq<sub>t</sub>* is the number of documents in the collection containing  term *t*.

### Document length

The impact of a document’s length is taken into account by multiplying the TF\*IDF score by $\frac{1}{\sqrt{|d|}}$

##### Lucene Lossy Compression

Lucene computes the lenght of a document |d| at indexing time, i.e. every time a document is added to the index. This information is then retrieved at search time, i.e. when the query is performed. 

However, to efficiently store this information, Lucene performs a lossy compression. The document length value is encoded as a single byte before being stored. At search time, the byte value is read from the index directory and decoded back to a float value. This encoding/decoding, while reducing index size, comes with the price of precision loss, i.e. it is not guaranteed that decode( encode(x) ) = x.

The rationale supporting such lossy compression is that given the difficulty (and inaccuracy) of users to express their true information need by a query, only big differences matter.

### Lucene Practical Scoring Function

Lucene's Practical Scoring Function (simplified version) is as follows: 

$$score_{q,\; d}   =   coord_{q, d} \;  \times \;  queryNorm_{q} \;  \times \; \sum_{t\;  in\; q} \left ( tf_{t, d}\;  \times \; idf_{t}^{2} \;  \times \;  norm_{d}  \right )$$

where:

* **coord<sub>q, d</sub>** : is a score factor based on how many of the query terms are found in the specified document. For example, if documnt *d* contains 2 of the 3 terms of query *q*, then coord<sub>q, d</sub> is 2/3.
* **queryNorm<sub>q</sub>** : is a normalizing factor used to make scores between queries comparable. This factor does not affect document ranking (since all ranked documents are multiplied by the same factor), but rather just attempts to make scores from different queries (or even different indexes) comparable. The default computation produces a Euclidean norm:

$$ queryNorm_{q} =\frac{1}{\sqrt{sumOfSquaredWeights}}   $$

    where

$$ sumOfSquaredWeights = \sum_{t \; in \; q} idf_{t}^{2} $$

* **tf<sub>t, d</sub>**: correlates to the term's frequency, defined as the number of times term t appears in the currently scored document d. Documents that have more occurrences of a given term receive a higher score. It is computed as explained above.

* **idf<sub>t</sub>** : stands for Inverse Document Frequency. This value correlates to the inverse of docFreq (the number of documents in which the term t appears). This means rarer terms give higher contribution to the total score. idf<sub>t</sub> appears for t in both the query and the document, hence it is squared in the equation. It is computed as explained above.

* **norm<sub>d</sub>** : encapsulates a (indexing time) length factor computed and stored as an 8 bit floating point value which causes some loss of precison (lossy compression).



### Simple Example

In [1]:
import os
import tqdm
import struct
import numpy as np
import pandas as pd
from scipy import stats
from scipy import sparse

from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Implement encoding and decoding function, as done here 
# https://github.com/apache/lucene-solr/blob/master/lucene/core/src/java/org/apache/lucene/util/SmallFloat.java
def floatToRawIntBits(f):
    s = struct.pack('=f', f)
    return struct.unpack('=l', s)[0]

def intBitsToFloat(b):
    s = struct.pack('>l', b)
    return struct.unpack('>f', s)[0]

def byte315ToFloat(b):
    if (b == 0):
        return 0.0
    bits = (b&0xff) << (24-3)
    bits += (63-15) << 24
    return intBitsToFloat(bits)

def floatToByte315(f):
    bits = floatToRawIntBits(f)
    
    smallfloat = bits >> (24-3)
    
    if (smallfloat <= ((63-15)<<3)):
        return  bytes(0) if (bits<=0) else bytes(1)
    
    if (smallfloat >= ((63-15)<<3) + 0x100):
        return -1
    
    return int(bytes(smallfloat - ((63-15)<<3)))

In [3]:
# Replicating the Lucene Classic Similarity Lossy Compression
NORM_TABLE = np.arange(256, dtype= float)
for i in range(256):
    NORM_TABLE[i] = byte315ToFloat(int(bytes(i)))

def decodeNormValue(b):
    return NORM_TABLE[b & 0xFF] # & 0xFF maps negative bytes to positive above 127

def encodeNormValue(fieldLength):
    return floatToByte315(fieldLength)

In [4]:
# Norm formula
def norm(value):
    length = 1.0/np.sqrt(value)
    return float(length)

In [5]:
def computeNorm(docs):
    vect = CountVectorizer(analyzer='word')
    X = vect.fit_transform(docs)
    docCount = X.shape[0]
    return np.matrix(map(decodeNormValue, map(encodeNormValue, map(norm, X.sum(axis = 1))))).reshape(docCount, 1)

In [6]:
def computeTFIDF(docs, vocab):
    # Document-term matrix for every term in the query
    vect = CountVectorizer(vocabulary = vocab, analyzer='word')
    tf = vect.fit_transform(docs)
    docCount = tf.shape[0]
    docFreqs = (tf != 0).sum(axis = 0)
    
    tf = tf.sqrt()
    
    idf = 1.0 + np.log(np.divide(docCount, (docFreqs + 1.0)))
    
    idf = np.square(idf, idf)
    
    lengthNorm = computeNorm(docs)
    
    tf = tf.multiply(lengthNorm)

    return (tf, idf.T)

In [22]:
def scores(query, docs):
    queryTerms = query.split(' ')
    tf, idf = computeTFIDF(docs, queryTerms)
    queryNorm = np.divide(1.0, np.sqrt(idf.sum(axis = 0)))
    idf = np.multiply(idf, queryNorm)
    res = tf.dot(idf)
    coord = np.divide((tf != 0).astype(float)*(idf != 0).astype(float), len(queryTerms))
    res = np.multiply(res, coord)
    return res  

In [23]:
docs = pd.Series(["Lucene Action continue continued", "Lucene  Dummies mbuy", "Managing Gigabytes", 
                  "Art  Computer Science", "Action", "Lucene way", "Managing Megabytes lucene", "Art Gaming"])

In [24]:
query = "action"
res = scores(query, docs)
res

matrix([[ 0.99041463],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 1.98082925],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]])

In [25]:
candidates = res != 0
indices = docs.index[np.asarray(candidates.T)[0,:]]
scores = np.asarray(res[indices].T)[0,:]
sorted_indices = indices[np.argsort(-scores)]
for rank, idx in enumerate(sorted_indices):
    print "%d.\t%s\t%f" %(rank, docs[idx], res[idx, 0])

0.	Action	1.980829
1.	Lucene Action continue continued	0.990415


## FieldBoosts and TermBoosts

The above example represents a simplified version of what Lucene can do. In fact, the formula can be enriched with **FieldBoosts** and **TermBoosts**. 

##### FieldBoost
Above, by document we meant a simple string of text. In reality, documents can be composed by multiple fields. For example, a book can heve the following fields: Title, Authors, Text.
With Lucene is possible to perform a query to multiple fields, or perform different queries to different fields.
The TDF-IDF scores are computed for each query-field pair and then summed together. Sometimes some fields might be more important then other, thus Lucene allows to specify a fieldBoost which can lower or increase the query-field score contribution to the final score. 

##### TermBoost
TermBoost is a search time boost of term t in the query q as specified in the query text. For example, a query specified this way "lucene way^0.2" will assign a weight of 1 (the default) to the term "lucene" and a weight of 0.2 to the term "way". TermBoosts can be used to lower (&lt;1) or increase (>1) the importance of the term in the query.

## Lucene Query Parser

Additional things Lucene does that here are not done is the parsing of the query string and of the documents. Lucene provides several query parsers. By default, the simple parser remove stopwords and apply case fold.

## New Lucene Default Similarity: BM25

BM25 stands for [**Best Matching 25**](https://en.wikipedia.org/wiki/Okapi_BM25), also called *Okapi weighting scheme*, and improves upon the classical TF \* IDF.

Released in [1994](http://trec.nist.gov/pubs/trec3/t3_proceedings.html), BM25 has its roots in [probabilistic information retrieval](http://nlp.stanford.edu/IR-book/html/htmledition/probabilistic-information-retrieval-1.html). A relevance score, according to probabilistic information retrieval, reflects the probability a user will consider the result relevant. 

The [BM25 weighting scheme](https://github.com/apache/lucene-solr/blob/master/lucene/core/src/java/org/apache/lucene/search/similarities/BM25Similarity.java) has become the default similarity measure from **Lucene 6**.

### Lucene BM25 view of IDF

The [original BM25 formula for IDF](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) is computed as:

$$idf_{t} = \ln (\frac{docCount - docFreq_{t} + 0.5}{docFreq_{t} + 0.5})$$

where *docCount* is the total number of documents in the collection, and *docFreq<sub>t</sub>* is the number of documents in the collection containing  term *t*.

It shows potentially major drawbacks when using it for terms appearing in more than half of the corpus documents. These terms' IDF is negative, so for any two almost-identical documents, one which contains the term and one which does not contain it, the latter will possibly get a larger score. This means that terms appearing in more than half of the corpus will provide negative contributions to the final document score. This is often an undesirable behavior, so Lucene overcomes this problem by adding 1 to the value, before taking the log, which makes it impossible to compute a negative value.

$$idf_{t} = \ln (1 + \frac{docCount - docFreq_{t} + 0.5}{docFreq_{t} + 0.5})$$




### Lucene BM25 view of TF and the influence of docLength

Term frequency in BM25 lower the impact of term frequency even further than traditional TF \* IDF. In the BM25, the impact of term frequency is always increasing, but asymptotically approaches the value (k + 1) (where in the default implementation k = 1.2).

$$ \frac{(k + 1) \times tf_{t, d}}{k + tf_{t, d}} $$

More *tf* always means more relevance. However in this way it quickly hits diminishing returns. Classic *tf*, on the other hand, constantly increases and never reaches a saturation point.

Changing k can be a useful tuning approach to modify the impact of the *tf*. A higher k causes *tf* to take longer to reach saturation. By stretching out the point of saturation, it stretches out the relevance difference between higher and lower term frequency docs.

The TF score above is further influenced by whether the document length is above or below the average length of a document in the corpus.
The formula above is modified by adding (1.0 - b + b * L) as a multiple on k in the denominator.

$$ tfNorm_{t,\; d} =  \frac{(k + 1) \times  tf_{t, d}}{k \times  (1.0 - b + b \times  L) + tf_{t, d}} $$

Here L is how long a document is relative to the average document length. L therefore is actually presented as $ \frac{|d|}{avgDocumentLength} $, i.e. this document length divided by the average document length.

The constant b (where in the default implementation b = 0.75) allows to finely tune how much influence the L value has on scoring. A b of 0 completely removes the influence of L. A higher b adds more document length influence on the scoring. 

### Lucene Lossy Compression

Lucene computes the lenght of a document |d| at indexing time, i.e. every time a document is added to the index. This information is then retrieved at search time, i.e. when the query is performed. 

However, to efficiently store this information, Lucene performs a lossy compression. The document length value is encoded as a single byte before being stored. At search time, the byte value is read from the index directory and decoded back to a float value. This encoding/decoding, while reducing index size, comes with the price of precision loss, i.e. it is not guaranteed that decode( encode(x) ) = x.

The rationale supporting such lossy compression is that given the difficulty (and inaccuracy) of users to express their true information need by a query, only big differences matter.



### All Together

Putting all together, the score of a document *d* given a query *q* (with multiple terms) is given by:

$$ score_{q,\; d} = \sum_{t\; in \; q }tfNorm_{t,\; d} \times  idf_{t} $$


### COICOP Simple Example

In [2]:
# Path variables
os.chdir("/Users/Alessandra/Downloads/foodies")
ROOT = os.getcwd()

In [3]:
# Load the data
strip = lambda x : x.strip()
foodies = pd.read_csv(os.path.join(ROOT, "clean2014Q3.csv") , sep = ",", header=0,
                      names=["coicop", "EXPDESC", "Paid1", "Shop", "MAFFQuan", "MAFFUnit"],
                      converters = {'coicop' : strip,
                                    'EXPDESC' : strip,
                                    'Paid1' : strip,
                                    'Shop' : strip,
                                    'MAFFQuan': float,
                                    'MAFFUnit': strip})

In [4]:
foodies.head()

,coicop,EXPDESC,Paid1,Shop,MAFFQuan,MAFFUnit
0,11111,loaf wht unsl fh,80,20,400.0,Grams
1,11111,loaf wht unsl fh,80,20,400.0,Grams
2,11111,loaf wht unsl fh,85,20,600.0,Grams
3,11111,wht bread un sl,100,20,400.0,Grams
4,11111,wht bread un sl,100,20,400.0,Grams


In [5]:
# Split training and tstind data
X_train, X_test, y_train, y_test = train_test_split(foodies["EXPDESC"], 
                                                    foodies["coicop"], 
                                                    test_size=0.2, random_state=10)

In [4]:
# A different lossy compression
NORM_TABLE = np.arange(256, dtype= float)
NORM_TABLE[0] = 0.0
for i in range(1, 256):
    f = byte315ToFloat(int(bytes(i)))
    NORM_TABLE[i] = 1.0 / (f*f)

NORM_TABLE[0] = 1.0 / NORM_TABLE[255]

def decodeNormValue(b):
    return NORM_TABLE[b & 0xFF]

def encodeNormValue(fieldLength):
    boost = 1.0
    return floatToByte315(boost / float(np.sqrt(fieldLength)))

In [61]:
# Compute documents length
def getFieldLength(docs, lossy):
    
    # Document-term matrix for every term in the documents
    vect = CountVectorizer(analyzer='word')
    X = vect.fit_transform(docs)
    docCount = X.shape[0]
    
    # Sum all counts / Number of docs
    avgFieldLength = X.sum()/float(docCount)
    
    # Encode and decode lengths for every document
    if lossy:
        lengths = map(encodeNormValue, X.sum(axis = 1))
        fieldLengths = np.matrix(map(decodeNormValue, lengths)).reshape(docCount, 1)
    else:
        fieldLengths = np.matrix(X.sum(axis = 1)).reshape(docCount, 1)
        
    
    return(fieldLengths, avgFieldLength)

In [74]:
# Compute idf, tfNorm
def get_tfidf(docs, vocab, lossy, k, b):
    
    # Document-term matrix for every term in the queries (X_test)
    vect = CountVectorizer(vocabulary = vocab, analyzer='word')
    tf = vect.fit_transform(docs)
    docCount = tf.shape[0]
    docFreqs = (tf != 0).sum(0)
    fieldLengths, avgFieldLength = getFieldLength(docs, lossy)
    
    # Compute idf, tfNorm
    idf = np.log(1 + np.divide((docCount - docFreqs + 0.5), (docFreqs + 0.5)))
    tfNorm = np.divide((tf * (k + 1)).toarray(),(tf + k * (1 - b + b * (fieldLengths/avgFieldLength))))
    
    return (tfNorm, idf.T)
    

In [9]:
# Return best neighbour class
def best(nn):
    mode = stats.mode(nn, axis=None, nan_policy='omit')
    return mode.mode[0]

In [112]:
def score(X_train, X_test, y_train, boosted = [], hintered = [], lossy = True, k = 1.2, b = 0.75):
    
    q = CountVectorizer(analyzer='word')
    queries = q.fit_transform(X_test)
    presences = (queries != 0).astype(int)
    vocab = q.get_feature_names()
    
    tfNorm, idf = get_tfidf(X_train, vocab, lossy, k, b)
    if boosted or hintered :
        idf_2 = np.multiply(idf, np.matrix([0.5 if x in hintered else 2 if x in boosted else 1 for x in vocab]).T)
    
    idf = presences.T.multiply(idf) 
    
    print "Computed tfNorm of shape: %d x %d (n° documents, n° queries terms)" % tfNorm.shape
    print "Computed idf of shape: %d x %d (n° queries terms, n° queries)" % idf.shape
    
    # Sparse representation
    idf = sparse.csr_matrix(idf)
    tfNorm = sparse.csr_matrix(tfNorm)
    
    # Too expensive to do as a matrix multiplication, thus do it sequentially
    coicops = []
    for idx in tqdm.tqdm(range(queries.shape[0])):
        res = tfNorm.dot(idf[:,idx])
        candidates = res != 0

        indices = X_train.index[candidates.T.toarray()[0]]
        scores = res[candidates.T.toarray()[0]].T.toarray()[0]

        assert(len(indices) == len(scores))

        if (len(indices) != 0):
            knn = indices[np.argsort(-scores)[:5]]
            # print knn
            coicops.append(best(y_train.loc[knn]))
        else:
            coicops.append('no data')
    return coicops

In [90]:
coicops = score(X_train, X_test, y_train)

Computed tfNorm of shape: 298820 x 4563 (n° documents, n° queries terms)
Computed idf of shape: 4563 x 74705 (n° queries terms, n° queries)


100%|██████████| 74705/74705 [20:50<00:00, 59.75it/s]


In [91]:
# 5 digits coicops metrics
print(metrics.classification_report(y_test, coicops))

             precision    recall  f1-score   support

      11111       0.93      0.93      0.93       217
      11112       0.98      0.99      0.99      1241
      11113       1.00      1.00      1.00         1
      11114       0.70      0.78      0.74        18
      11121       0.97      0.96      0.97       163
      11122       0.98      0.98      0.98       879
      11131       0.98      0.98      0.98       955
      11132       0.95      0.92      0.94        90
      11133       0.94      0.97      0.95       318
      11134       0.89      1.00      0.94        58
      11135       1.00      0.97      0.99       252
      11136       0.96      0.94      0.95       953
      11141       0.95      0.98      0.96       803
      11142       0.95      0.97      0.96       183
      11143       0.96      0.95      0.96      1197
      11144       0.97      0.97      0.97      1808
      11145       0.97      0.93      0.95       326
      11151       0.95      0.95      0.95   

In [92]:
# 4 digits coicops metrics
print(metrics.classification_report([x[:4] for x in y_test], 
                                    [x[:4] for x in coicops]))

             precision    recall  f1-score   support

       1111       0.98      0.99      0.99      1477
       1112       0.98      0.98      0.98      1042
       1113       0.98      0.97      0.98      2626
       1114       0.98      0.98      0.98      4317
       1115       0.98      0.98      0.98      1297
       1116       0.97      0.96      0.97      2446
       1117       0.96      0.97      0.97      2798
       1118       0.98      0.96      0.97       840
       1121       0.98      0.99      0.99       931
       1122       0.98      0.97      0.98       386
       1123       1.00      0.99      1.00       780
       1124       0.97      0.98      0.97       218
       1125       0.99      0.99      0.99      1162
       1126       0.96      0.93      0.95        59
       1127       1.00      0.82      0.90        11
       1128       0.98      0.98      0.98      1113
       1129       0.92      0.97      0.94       146
       1131       0.95      0.92      0.94   